In [31]:
import jenkspy
import pandas as pd

In [32]:
df_11_12 = pd.read_csv('dissimilarity_11_12.csv', dtype={'LEAID':str})
df_13_14 = pd.read_csv('dissimilarity_13_14.csv', dtype={'LEAID':str})
df_15_16 = pd.read_csv('dissimilarity_15_16.csv', dtype={'LEAID':str})

In [33]:
def classify(x, l1, l2):
    if x <= l1:
        return 'low'
    if x <= l2:
        return 'med'
    if x < 1.0:
        return 'high'

In [34]:
def categorize_dscore(df_column):
    list_of_values = df_column
    breaks = jenkspy.jenks_breaks(list_of_values, nb_class=6)
    lim1 = breaks[2]
    lim2 = breaks[4]
    return df_column.apply(lambda x: classify(x, lim1, lim2))

In [35]:
frames = [df_11_12, df_13_14, df_15_16]

In [36]:
for df in frames:
    for col in df.columns:
        if 'd_' in col:
            df[f'classified_{col}'] = categorize_dscore(df[col])